In [109]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import load_data, get_label_dict, get_word_labels

from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os
import keras.backend as K

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
X_train, y_train = load_data()
X_val, y_val = load_data('val')
X_test, _ = load_data('test')

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading data

Loading val data

Finished loading data

Loading test data

Finished loading data



In [111]:
labels = get_label_dict()
test_images = os.listdir('data/tiny-imagenet-200/test/images/')
words = get_word_labels()

Loading labels

Done

Loading words

Done



In [112]:
def show_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [113]:
datagen = ImageDataGenerator(rotation_range=20,
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             horizontal_flip=True, 
                             rescale=1./255)

testgen = ImageDataGenerator(rescale=1./255)

datagen.fit(X_train)

testgen.fit(X_val)
testgen.fit(X_test)

data = {'train': (X_train, y_train), 
        'val': (X_val, y_val)}

In [114]:
model = AlphaNet()

In [117]:
checkpoint = ModelCheckpoint('models/' + model.name + '/' + model.name + ".h5", 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='auto', 
                             period=1)

cb_list = [checkpoint]

In [119]:
history = train(restore=False, 
                epochs=400,
                model=model, 
                datagen=datagen,
                data=data, 
                cb_list=cb_list)

Model: "AlphaNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_57 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu_57 (LeakyReLU)   (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
spatial_dropout2d_33 (Spatia (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 32, 32, 64)        51264     
_________________________________________________________________
batch_normalization_58 (Batc (None, 32, 32, 64)        256

KeyboardInterrupt: 

In [ ]:
show_history(history)

In [ ]:
output = model.predict_generator(testgen.flow(X_test, shuffle=False))
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(test_images)
print("Test images predicted:", len(results))

In [ ]:
with open('models/' + model.name + 
          '/' + model.name + ' Predictions.txt', 'w') as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + ' ' + results[i] + '\n')

In [ ]:
for _ in range(10):
    index = np.random.randint(10000)
    img = Image.fromarray(X_test[index], 'RGB').resize(size=(128, 128))
    word_label = words[results[index]]
    display(img)
    print(word_label)